In [ ]:
import pandas as pd

# Read the CSV file
trec_data = pd.read_csv("train.csv")

# View the first 5 rows
trec_data.head()

,label-coarse,label-fine,text
0,0,0,How did serfdom develop in and then leave Russ...
1,1,1,What films featured the character Popeye Doyle ?
2,0,0,How can I find a list of celebrities ' real na...
3,1,2,What fowl grabs the spotlight after the Chines...
4,2,3,What is the full form of .com ?


In [ ]:
import random
random_4_labels = random.sample(range(5), 4)
print(random_4_labels)

[4, 2, 0, 3]


In [ ]:
trec_updated_labels_df = trec_data

def replace_label_coarse(value):
    if value not in random_4_labels:
        return 'OTHERS' #I use integer 10 to represent 'OTHERS'
    return value

trec_updated_labels_df['label-coarse'] = trec_updated_labels_df['label-coarse'].apply(replace_label_coarse)
trec_updated_labels_df['label-coarse'].unique()

array([0, 'OTHERS', 2, 3, 4], dtype=object)

In [ ]:
trec_updated_labels_df.head()

,label-coarse,label-fine,text
0,0,0,How did serfdom develop in and then leave Russ...
1,OTHERS,1,What films featured the character Popeye Doyle ?
2,0,0,How can I find a list of celebrities ' real na...
3,OTHERS,2,What fowl grabs the spotlight after the Chines...
4,2,3,What is the full form of .com ?


In [ ]:
classes = trec_updated_labels_df['label-coarse'].unique()
print(classes)
num_classes = len(classes)
print(num_classes)

[0 'OTHERS' 2 3 4]
5


In [ ]:
di = {}
idx = 0
for c in classes:
    di[c] = idx
    idx+=1

trec_updated_labels_df.replace({"label-coarse": di}, inplace=True)
trec_updated_labels_df['label-coarse'].unique()
print("Label mapping: ", di)

Label mapping:  {0: 0, 'OTHERS': 1, 2: 2, 3: 3, 4: 4}


In [ ]:
from sklearn.model_selection import train_test_split

train, dev = train_test_split(trec_updated_labels_df, test_size=(500/trec_data.shape[0]))

In [ ]:
train.shape

(4952, 3)

In [ ]:
dev.shape

(500, 3)

In [ ]:
test = pd.read_csv("test.csv")
test['label-coarse'] = test['label-coarse'].apply(replace_label_coarse)
test.replace({"label-coarse": di}, inplace=True)
test['label-coarse'].unique()

array([4, 1, 3, 0, 2])

In [ ]:
test.head()

,label-coarse,label-fine,text
0,4,40,How far is it from Denver to Aspen ?
1,1,21,"What county is Modesto , California in ?"
2,3,12,Who was Galileo ?
3,0,7,What is an atom ?
4,4,8,When did Hawaii become a state ?


In [ ]:
test.shape

(500, 3)

In [ ]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, GlobalAveragePooling1D, Dense
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.callbacks import EarlyStopping, ModelCheckpoint
from gensim.models import Word2Vec
import gensim.downloader

# Load and preprocess the TREC dataset
# (Assuming you have loaded the dataset into train_df, dev_df, and test_df)

# Load the pretrained Word2Vec model
word2vec_model = gensim.downloader.load('word2vec-google-news-300')

2023-10-17 04:57:52.891623: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-17 04:57:52.913292: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-17 04:57:53.003568: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-17 04:57:53.003596: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-17 04:57:53.004213: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [ ]:
# Tokenize and create sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train['text'])
X_train = tokenizer.texts_to_sequences(train['text'])
X_dev = tokenizer.texts_to_sequences(dev['text'])
X_test = tokenizer.texts_to_sequences(test['text'])

In [ ]:
# Pad sequences
max_seq_length = 200  # You can adjust this
X_train = pad_sequences(X_train, maxlen=max_seq_length)
X_dev = pad_sequences(X_dev, maxlen=max_seq_length)
X_test = pad_sequences(X_test, maxlen=max_seq_length)

# Load labels and convert to one-hot encoding
num_classes = 5
y_train = to_categorical(train['label-coarse'], num_classes=num_classes)
y_dev = to_categorical(dev['label-coarse'], num_classes=num_classes)
y_test = to_categorical(test['label-coarse'], num_classes=num_classes)

# Exp 1: LSTM with Global Average Pooling

In [ ]:
# Create the LSTM model
model = Sequential()

2023-10-17 03:26:17.098097: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-17 03:26:17.153222: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
# Add an embedding layer with pretrained word embeddings
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, 300))  # Adjust embedding dimension
for word, i in tokenizer.word_index.items():
    if word in word2vec_model:
        embedding_matrix[i] = word2vec_model[word]
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_seq_length, trainable=False)
model.add(embedding_layer)

In [ ]:
# Add an LSTM layer
model.add(LSTM(100, return_sequences=True))

# Add Global Average Pooling1D for aggregation
model.add(GlobalAveragePooling1D())

# Add the output layer
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
# Training with early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

history = model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=100, batch_size=32, callbacks=[early_stopping, model_checkpoint])

Epoch 1/100
155/155 [==============================] - 10s 58ms/step - loss: 1.4031 - accuracy: 0.3760 - val_loss: 1.3414 - val_accuracy: 0.3760
Epoch 2/100
  1/155 [..............................] - ETA: 8s - loss: 1.3510 - accuracy: 0.4375

/home/grace/anaconda3/envs/nlp2/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


155/155 [==============================] - 9s 56ms/step - loss: 1.2260 - accuracy: 0.4707 - val_loss: 1.1702 - val_accuracy: 0.4920
Epoch 3/100
155/155 [==============================] - 9s 56ms/step - loss: 1.0709 - accuracy: 0.6254 - val_loss: 1.0717 - val_accuracy: 0.6140
Epoch 4/100
155/155 [==============================] - 9s 56ms/step - loss: 0.9598 - accuracy: 0.7151 - val_loss: 0.9923 - val_accuracy: 0.6340
Epoch 5/100
155/155 [==============================] - 9s 57ms/step - loss: 0.8746 - accuracy: 0.7510 - val_loss: 0.9169 - val_accuracy: 0.7380
Epoch 6/100
155/155 [==============================] - 9s 55ms/step - loss: 0.8062 - accuracy: 0.7851 - val_loss: 0.8654 - val_accuracy: 0.7700
Epoch 7/100
155/155 [==============================] - 9s 57ms/step - loss: 0.7435 - accuracy: 0.8067 - val_loss: 0.8204 - val_accuracy: 0.8000
Epoch 8/100
155/155 [==============================] - 9s 55ms/step - loss: 0.6957 - accuracy: 0.8277 - val_loss: 0.7969 - val_accuracy: 0.7900


In [ ]:
# Evaluate on the test set
test_predictions = model.predict(X_test)
test_predictions = np.argmax(test_predictions, axis=-1)
test_accuracy = accuracy_score(np.argmax(y_test, axis=-1), test_predictions)
print(f"Test Accuracy: {test_accuracy:.4f}")

16/16 [==============================] - 1s 25ms/step
Test Accuracy: 0.7980


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 300)          2397300   
                                                                 
 lstm (LSTM)                 (None, 200, 100)          160400    
                                                                 
 global_average_pooling1d (  (None, 100)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 5)                 505       
                                                                 
Total params: 2558205 (9.76 MB)
Trainable params: 160905 (628.54 KB)
Non-trainable params: 2397300 (9.14 MB)
_________________________________________________________________


# Exp 2: LSTM with Weighted Pooling

In [ ]:
# Create the LSTM model
model = Sequential()

# Add an embedding layer with pretrained word embeddings
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, 300))  # Adjust embedding dimension
for word, i in tokenizer.word_index.items():
    if word in word2vec_model:
        embedding_matrix[i] = word2vec_model[word]
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_seq_length, trainable=False)
model.add(embedding_layer)

# Add an LSTM layer
model.add(LSTM(100, return_sequences=True))

In [ ]:
# Try using weighted pooling
from keras.layers import Layer
import tensorflow as tf

class WeightedPooling(Layer):
    def __init__(self, **kwargs):
        super(WeightedPooling, self).__init__(**kwargs)

    def build(self, input_shape):
        self.custom_weights = self.add_weight(name='custom_weights', shape=(input_shape[-1],), initializer='ones', trainable=True)
        super(WeightedPooling, self).build(input_shape)

    def call(self, x):
        weighted_x = x * self.custom_weights
        return tf.reduce_sum(weighted_x, axis=1)

# Use the custom WeightedPooling layer in your model
model.add(WeightedPooling())

In [ ]:
# Add the output layer
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Training with early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

history = model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=100, batch_size=32, callbacks=[early_stopping, model_checkpoint])

Epoch 1/100
155/155 [==============================] - 10s 59ms/step - loss: 0.8946 - accuracy: 0.6460 - val_loss: 0.6865 - val_accuracy: 0.7100
Epoch 2/100
  3/155 [..............................] - ETA: 7s - loss: 0.8825 - accuracy: 0.6979 

/home/grace/anaconda3/envs/nlp2/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


155/155 [==============================] - 9s 55ms/step - loss: 0.5572 - accuracy: 0.7971 - val_loss: 0.5602 - val_accuracy: 0.7860
Epoch 3/100
155/155 [==============================] - 9s 56ms/step - loss: 0.4306 - accuracy: 0.8391 - val_loss: 0.4545 - val_accuracy: 0.8460
Epoch 4/100
155/155 [==============================] - 9s 56ms/step - loss: 0.3518 - accuracy: 0.8738 - val_loss: 0.4085 - val_accuracy: 0.8600
Epoch 5/100
155/155 [==============================] - 9s 56ms/step - loss: 0.3038 - accuracy: 0.8891 - val_loss: 0.3966 - val_accuracy: 0.8600


In [ ]:
# Evaluate on the test set
test_predictions = model.predict(X_test)
test_predictions = np.argmax(test_predictions, axis=-1)
test_accuracy = accuracy_score(np.argmax(y_test, axis=-1), test_predictions)
print(f"Test Accuracy: {test_accuracy:.4f}")

16/16 [==============================] - 1s 23ms/step
Test Accuracy: 0.8360


# Exp 3: LSTM with GAP. Change loss function to categorical crossentropy with label smoothing

In [ ]:
# Create the LSTM model
model = Sequential()

# Add an embedding layer with pretrained word embeddings
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, 300))  # Adjust embedding dimension
for word, i in tokenizer.word_index.items():
    if word in word2vec_model:
        embedding_matrix[i] = word2vec_model[word]
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_seq_length, trainable=False)
model.add(embedding_layer)

# Add an LSTM layer
model.add(LSTM(100, return_sequences=True))

# Add Global Average Pooling1D for aggregation
model.add(GlobalAveragePooling1D())

# Add the output layer
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
def categorical_crossentropy_with_label_smoothing(y_true, y_pred, label_smoothing=0.1):
    y_true_smoothed = (1 - label_smoothing) * y_true + label_smoothing / num_classes  # Assuming num_classes is defined
    return tf.keras.losses.categorical_crossentropy(y_true_smoothed, y_pred)

# Compile your model with the custom loss function
model.compile(loss=categorical_crossentropy_with_label_smoothing, optimizer=Adam(), metrics=['accuracy'])

In [ ]:
# Training with early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

history = model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=100, batch_size=32, callbacks=[early_stopping, model_checkpoint])

Epoch 1/100
155/155 [==============================] - 10s 58ms/step - loss: 1.4477 - accuracy: 0.3766 - val_loss: 1.3944 - val_accuracy: 0.3820
Epoch 2/100
  3/155 [..............................] - ETA: 7s - loss: 1.3915 - accuracy: 0.3646

/home/grace/anaconda3/envs/nlp2/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


155/155 [==============================] - 9s 57ms/step - loss: 1.3076 - accuracy: 0.4814 - val_loss: 1.2668 - val_accuracy: 0.6020
Epoch 3/100
155/155 [==============================] - 9s 57ms/step - loss: 1.1772 - accuracy: 0.6468 - val_loss: 1.1653 - val_accuracy: 0.7040
Epoch 4/100
155/155 [==============================] - 9s 56ms/step - loss: 1.0842 - accuracy: 0.7155 - val_loss: 1.1031 - val_accuracy: 0.7000


In [ ]:
# Evaluate on the test set
test_predictions = model.predict(X_test)
test_predictions = np.argmax(test_predictions, axis=-1)
test_accuracy = accuracy_score(np.argmax(y_test, axis=-1), test_predictions)
print(f"Test Accuracy: {test_accuracy:.4f}")

16/16 [==============================] - 1s 50ms/step
Test Accuracy: 0.4940


# Exp 4: LSTM with Weighted Pooling. Change loss function to categorical crossentropy with label smoothing

In [ ]:
# Create the LSTM model
model = Sequential()

# Add an embedding layer with pretrained word embeddings
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, 300))  # Adjust embedding dimension
for word, i in tokenizer.word_index.items():
    if word in word2vec_model:
        embedding_matrix[i] = word2vec_model[word]
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_seq_length, trainable=False)
model.add(embedding_layer)

# Add an LSTM layer
model.add(LSTM(100, return_sequences=True))

from keras.layers import Layer
import tensorflow as tf

class WeightedPooling(Layer):
    def __init__(self, **kwargs):
        super(WeightedPooling, self).__init__(**kwargs)

    def build(self, input_shape):
        self.custom_weights = self.add_weight(name='custom_weights', shape=(input_shape[-1],), initializer='ones', trainable=True)
        super(WeightedPooling, self).build(input_shape)

    def call(self, x):
        weighted_x = x * self.custom_weights
        return tf.reduce_sum(weighted_x, axis=1)

# Use the custom WeightedPooling layer in your model
model.add(WeightedPooling())

# Add the output layer
model.add(Dense(num_classes, activation='softmax'))

def categorical_crossentropy_with_label_smoothing(y_true, y_pred, label_smoothing=0.1):
    y_true_smoothed = (1 - label_smoothing) * y_true + label_smoothing / num_classes  # Assuming num_classes is defined
    return tf.keras.losses.categorical_crossentropy(y_true_smoothed, y_pred)

# Compile your model with the custom loss function
model.compile(loss=categorical_crossentropy_with_label_smoothing, optimizer=Adam(), metrics=['accuracy'])

# Training with early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

history = model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=100, batch_size=32, callbacks=[early_stopping, model_checkpoint])

Epoch 1/100
155/155 [==============================] - 10s 57ms/step - loss: 1.0639 - accuracy: 0.6488 - val_loss: 0.9000 - val_accuracy: 0.7700
Epoch 2/100
  2/155 [..............................] - ETA: 9s - loss: 0.7414 - accuracy: 0.8281

/home/grace/anaconda3/envs/nlp2/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


155/155 [==============================] - 9s 56ms/step - loss: 0.8465 - accuracy: 0.7849 - val_loss: 0.8266 - val_accuracy: 0.7940
Epoch 3/100
155/155 [==============================] - 9s 56ms/step - loss: 0.7478 - accuracy: 0.8389 - val_loss: 0.7531 - val_accuracy: 0.8380
Epoch 4/100
155/155 [==============================] - 8s 54ms/step - loss: 0.6762 - accuracy: 0.8760 - val_loss: 0.7429 - val_accuracy: 0.8240


In [ ]:
# Evaluate on the test set
test_predictions = model.predict(X_test)
test_predictions = np.argmax(test_predictions, axis=-1)
test_accuracy = accuracy_score(np.argmax(y_test, axis=-1), test_predictions)
print(f"Test Accuracy: {test_accuracy:.4f}")

16/16 [==============================] - 1s 25ms/step
Test Accuracy: 0.8640


# Exp 5: LSTM with Weighted Pooling. Change loss function to focal loss

In [ ]:
# Create the LSTM model
model = Sequential()

# Add an embedding layer with pretrained word embeddings
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, 300))  # Adjust embedding dimension
for word, i in tokenizer.word_index.items():
    if word in word2vec_model:
        embedding_matrix[i] = word2vec_model[word]
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_seq_length, trainable=False)
model.add(embedding_layer)

# Add an LSTM layer
model.add(LSTM(100, return_sequences=True))

from keras.layers import Layer
import tensorflow as tf

class WeightedPooling(Layer):
    def __init__(self, **kwargs):
        super(WeightedPooling, self).__init__(**kwargs)

    def build(self, input_shape):
        self.custom_weights = self.add_weight(name='custom_weights', shape=(input_shape[-1],), initializer='ones', trainable=True)
        super(WeightedPooling, self).build(input_shape)

    def call(self, x):
        weighted_x = x * self.custom_weights
        return tf.reduce_sum(weighted_x, axis=1)

# Use the custom WeightedPooling layer in your model
model.add(WeightedPooling())

# Add the output layer
model.add(Dense(num_classes, activation='softmax'))

def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        pt = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        focal_loss = -alpha * (1 - pt)**gamma * tf.math.log(pt)
        return focal_loss

    return focal_loss_fixed

# Compile your model with the custom loss function
model.compile(loss=focal_loss(), optimizer=Adam(), metrics=['accuracy'])

# Training with early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

history = model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=100, batch_size=32, callbacks=[early_stopping, model_checkpoint])

Epoch 1/100
155/155 [==============================] - 10s 58ms/step - loss: 0.0290 - accuracy: 0.6349 - val_loss: 0.0242 - val_accuracy: 0.6520
Epoch 2/100
  3/155 [..............................] - ETA: 8s - loss: 0.0189 - accuracy: 0.7083

/home/grace/anaconda3/envs/nlp2/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


155/155 [==============================] - 9s 56ms/step - loss: 0.0177 - accuracy: 0.7748 - val_loss: 0.0200 - val_accuracy: 0.7460
Epoch 3/100
155/155 [==============================] - 9s 56ms/step - loss: 0.0139 - accuracy: 0.8273 - val_loss: 0.0146 - val_accuracy: 0.8280
Epoch 4/100
155/155 [==============================] - 9s 55ms/step - loss: 0.0112 - accuracy: 0.8526 - val_loss: 0.0150 - val_accuracy: 0.7940


In [ ]:
# Evaluate on the test set
test_predictions = model.predict(X_test)
test_predictions = np.argmax(test_predictions, axis=-1)
test_accuracy = accuracy_score(np.argmax(y_test, axis=-1), test_predictions)
print(f"Test Accuracy: {test_accuracy:.4f}")

16/16 [==============================] - 1s 24ms/step
Test Accuracy: 0.8180


# Exp 6: Bidirectional LSTM with GAP.

In [ ]:
from keras.layers import Bidirectional
# Create the LSTM model
model = Sequential()

# Add an embedding layer with pretrained word embeddings
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_seq_length, trainable=False)
model.add(embedding_layer)

# Add an LSTM layer
model.add(Bidirectional(LSTM(100, return_sequences=True)))

# Add another LSTM layer
model.add(Bidirectional(LSTM(100, return_sequences=True)))

# Add Global Average Pooling1D for aggregation
model.add(GlobalAveragePooling1D())

# Add the output layer
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Training with early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

history = model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=100, batch_size=32, callbacks=[early_stopping, model_checkpoint])

Epoch 1/100
155/155 [==============================] - 21s 119ms/step - loss: 1.2168 - accuracy: 0.4885 - val_loss: 0.9758 - val_accuracy: 0.6200
Epoch 2/100
  1/155 [..............................] - ETA: 21s - loss: 0.8929 - accuracy: 0.5938

/home/grace/anaconda3/envs/nlp2/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


155/155 [==============================] - 18s 117ms/step - loss: 0.9366 - accuracy: 0.6440 - val_loss: 0.9713 - val_accuracy: 0.5580


In [ ]:
# Evaluate on the test set
test_predictions = model.predict(X_test)
test_predictions = np.argmax(test_predictions, axis=-1)
test_accuracy = accuracy_score(np.argmax(y_test, axis=-1), test_predictions)
print(f"Test Accuracy: {test_accuracy:.4f}")

16/16 [==============================] - 2s 44ms/step
Test Accuracy: 0.6200


# Exp 7: Bidirectional LSTM with Weighted Pooling

In [ ]:
from keras.layers import Bidirectional
# Create the LSTM model
model = Sequential()

# Add an embedding layer with pretrained word embeddings
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_seq_length, trainable=False)
model.add(embedding_layer)

# Add an LSTM layer
model.add(Bidirectional(LSTM(100, return_sequences=True)))

# Add another LSTM layer
model.add(Bidirectional(LSTM(100, return_sequences=True)))

# Add Global Average Pooling1D for aggregation
model.add(WeightedPooling())

# Add the output layer
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Training with early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

history = model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=100, batch_size=32, callbacks=[early_stopping, model_checkpoint])

Epoch 1/100
155/155 [==============================] - 24s 127ms/step - loss: 0.9504 - accuracy: 0.6393 - val_loss: 0.7915 - val_accuracy: 0.6860
Epoch 2/100
  1/155 [..............................] - ETA: 19s - loss: 0.5052 - accuracy: 0.8438

/home/grace/anaconda3/envs/nlp2/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


155/155 [==============================] - 19s 125ms/step - loss: 0.5784 - accuracy: 0.7904 - val_loss: 0.4921 - val_accuracy: 0.8100
Epoch 3/100
155/155 [==============================] - 19s 123ms/step - loss: 0.4320 - accuracy: 0.8471 - val_loss: 0.4487 - val_accuracy: 0.8120
Epoch 4/100
155/155 [==============================] - 19s 124ms/step - loss: 0.3470 - accuracy: 0.8750 - val_loss: 0.5850 - val_accuracy: 0.7700


In [ ]:
# Evaluate on the test set
test_predictions = model.predict(X_test)
test_predictions = np.argmax(test_predictions, axis=-1)
test_accuracy = accuracy_score(np.argmax(y_test, axis=-1), test_predictions)
print(f"Test Accuracy: {test_accuracy:.4f}")

16/16 [==============================] - 1s 46ms/step
Test Accuracy: 0.8380


# Exp 8: Bidirectional LSTM with Weighted Pooling. Change loss function to categorical crossentropy with label smoothing

In [ ]:
from keras.layers import Bidirectional
# Create the LSTM model
model = Sequential()

# Add an embedding layer with pretrained word embeddings
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_seq_length, trainable=False)
model.add(embedding_layer)

# Add an LSTM layer
model.add(Bidirectional(LSTM(100, return_sequences=True)))

# Add Global Average Pooling1D for aggregation
model.add(WeightedPooling())

# Add the output layer
model.add(Dense(num_classes, activation='softmax'))

def categorical_crossentropy_with_label_smoothing(y_true, y_pred, label_smoothing=0.1):
    y_true_smoothed = (1 - label_smoothing) * y_true + label_smoothing / num_classes  # Assuming num_classes is defined
    return tf.keras.losses.categorical_crossentropy(y_true_smoothed, y_pred)

# Compile your model with the custom loss function
model.compile(loss=categorical_crossentropy_with_label_smoothing, optimizer=Adam(), metrics=['accuracy'])

# Training with early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

history = model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=100, batch_size=32, callbacks=[early_stopping, model_checkpoint])

Epoch 1/100
155/155 [==============================] - 14s 69ms/step - loss: 1.0973 - accuracy: 0.6468 - val_loss: 0.8646 - val_accuracy: 0.7660
Epoch 2/100
  2/155 [..............................] - ETA: 12s - loss: 0.9564 - accuracy: 0.7500

/home/grace/anaconda3/envs/nlp2/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


155/155 [==============================] - 10s 66ms/step - loss: 0.8385 - accuracy: 0.7880 - val_loss: 0.8325 - val_accuracy: 0.8040
Epoch 3/100
155/155 [==============================] - 11s 71ms/step - loss: 0.7406 - accuracy: 0.8461 - val_loss: 0.7545 - val_accuracy: 0.8480
Epoch 4/100
155/155 [==============================] - 11s 70ms/step - loss: 0.6919 - accuracy: 0.8722 - val_loss: 0.7701 - val_accuracy: 0.8080


In [ ]:
# Evaluate on the test set
test_predictions = model.predict(X_test)
test_predictions = np.argmax(test_predictions, axis=-1)
test_accuracy = accuracy_score(np.argmax(y_test, axis=-1), test_predictions)
print(f"Test Accuracy: {test_accuracy:.4f}")

16/16 [==============================] - 1s 24ms/step
Test Accuracy: 0.8580


# Exp 9: Bidirectional LSTM with Weighted Pooling and Self-Attention

In [ ]:
from keras.layers import Bidirectional
from keras_self_attention import SeqSelfAttention
# Create the LSTM model
model = Sequential()

# Add an embedding layer with pretrained word embeddings
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_seq_length, trainable=False)
model.add(embedding_layer)

# Add an LSTM layer
model.add(Bidirectional(LSTM(100, return_sequences=True)))

# Add another LSTM layer
model.add(Bidirectional(LSTM(100, return_sequences=True)))

# Add self-attention mechanism
model.add(SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL, attention_activation='sigmoid'))

# Add Global Average Pooling1D for aggregation
model.add(WeightedPooling())

# Add the output layer
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Training with early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

history = model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=100, batch_size=32, callbacks=[early_stopping, model_checkpoint])

Epoch 1/100
155/155 [==============================] - 26s 137ms/step - loss: 0.9569 - accuracy: 0.6387 - val_loss: 0.6443 - val_accuracy: 0.7560
Epoch 2/100
  1/155 [..............................] - ETA: 19s - loss: 0.4628 - accuracy: 0.8438

/home/grace/anaconda3/envs/nlp2/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


155/155 [==============================] - 21s 134ms/step - loss: 0.5634 - accuracy: 0.7944 - val_loss: 0.4906 - val_accuracy: 0.8420
Epoch 3/100
155/155 [==============================] - 21s 134ms/step - loss: 0.4142 - accuracy: 0.8548 - val_loss: 0.4997 - val_accuracy: 0.8000


In [ ]:
# Evaluate on the test set
test_predictions = model.predict(X_test)
test_predictions = np.argmax(test_predictions, axis=-1)
test_accuracy = accuracy_score(np.argmax(y_test, axis=-1), test_predictions)
print(f"Test Accuracy: {test_accuracy:.4f}")

16/16 [==============================] - 1s 50ms/step
Test Accuracy: 0.8620


# Exp 10: Bidirectional LSTM with Weighted Pooling and Self-Attention. Change loss function to categorical crossentropy with label smoothing

In [ ]:
from keras.layers import Bidirectional
from keras_self_attention import SeqSelfAttention
# Create the LSTM model
model = Sequential()

# Add an embedding layer with pretrained word embeddings
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, 300))  # Adjust embedding dimension
for word, i in tokenizer.word_index.items():
    if word in word2vec_model:
        embedding_matrix[i] = word2vec_model[word]
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_seq_length, trainable=False)
model.add(embedding_layer)

# Add an LSTM layer
model.add(Bidirectional(LSTM(100, return_sequences=True)))

# Add another LSTM layer
model.add(Bidirectional(LSTM(100, return_sequences=True)))

# Add self-attention mechanism
model.add(SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL, attention_activation='sigmoid'))

# Add weighted pooling
from keras.layers import Layer
import tensorflow as tf

class WeightedPooling(Layer):
    def __init__(self, **kwargs):
        super(WeightedPooling, self).__init__(**kwargs)

    def build(self, input_shape):
        self.custom_weights = self.add_weight(name='custom_weights', shape=(input_shape[-1],), initializer='ones', trainable=True)
        super(WeightedPooling, self).build(input_shape)

    def call(self, x):
        weighted_x = x * self.custom_weights
        return tf.reduce_sum(weighted_x, axis=1)

model.add(WeightedPooling())

# Add the output layer
model.add(Dense(num_classes, activation='softmax'))

# Compile your model with the custom loss function
def categorical_crossentropy_with_label_smoothing(y_true, y_pred, label_smoothing=0.1):
    y_true_smoothed = (1 - label_smoothing) * y_true + label_smoothing / num_classes  # Assuming num_classes is defined
    return tf.keras.losses.categorical_crossentropy(y_true_smoothed, y_pred)

model.compile(loss=categorical_crossentropy_with_label_smoothing, optimizer=Adam(), metrics=['accuracy'])

# Training with early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=1, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

history = model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=100, batch_size=32, callbacks=[early_stopping, model_checkpoint])

2023-10-17 05:01:38.414792: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-17 05:01:38.552694: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/100
155/155 [==============================] - 25s 127ms/step - loss: 1.1220 - accuracy: 0.6315 - val_loss: 0.8748 - val_accuracy: 0.7620
Epoch 2/100
  1/155 [..............................] - ETA: 19s - loss: 0.7805 - accuracy: 0.8750

/home/grace/anaconda3/envs/nlp2/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


155/155 [==============================] - 20s 126ms/step - loss: 0.8042 - accuracy: 0.8122 - val_loss: 0.9848 - val_accuracy: 0.7120


In [ ]:
# Evaluate on the test set
test_predictions = model.predict(X_test)
test_predictions = np.argmax(test_predictions, axis=-1)
test_accuracy = accuracy_score(np.argmax(y_test, axis=-1), test_predictions)
print(f"Test Accuracy: {test_accuracy:.4f}")

16/16 [==============================] - 1s 49ms/step
Test Accuracy: 0.7840


# Summary

### LSTM with Global Average Pooling, Categorical Cross Entropy Loss
Val accuracy: 0.800
Test Accuracy: 0.7980

### LSTM with Weighted Pooling, Categorical Cross Entropy Loss
Val accuracy: 0.860
Test Accuracy: 0.836

### LSTM with Global Average Pooling, Categorical Cross Entropy Loss with Label Smoothing
Val accuracy: 0.7040
Test Accuracy: 0.4940

### LSTM with Weighted Pooling, Categorical Cross Entropy Loss with Label Smoothing
Val accuracy: 0.8380
Test Accuracy: 0.8640

### LSTM with Weighted Pooling, Focal Loss
Val accuracy: 0.8280
Test Accuracy: 0.8180

### Bidirectional LSTM with Global Average Pooling, Categorical Cross Entropy Loss
Val accuracy: 0.6200
Test Accuracy: 0.6200

### Bidirectional LSTM with Weighted Pooling, Categorical Cross Entropy Loss
Val accuracy: 0.8120
Test Accuracy: 0.8380

### Bidirectional LSTM with Weighted Pooling, Categorical Cross Entropy Loss with Label Smoothing
Val accuracy: 0.8480
Test Accuracy: 0.8580

### Bidirectional LSTM with Weighted Pooling and Self-Attention, Categorical Cross Entropy Loss
Val accuracy: 0.8420
Test Accuracy: 0.8620

### Bidirectional LSTM with Weighted Pooling and Self-Attention, Categorical Cross Entropy Loss with Label Smoothing
Val accuracy: 0.7620
Test Accuracy: 0.7840